<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/digitaloption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import QuantLib as ql
import unittest
import math

# Helper utilities from C++ test
def flat_rate_py(evaluation_date, forward_rate, day_counter): # Name clash with ql.flatRate
    # Ensure q_rate_handle is a Handle<Quote>
    if isinstance(forward_rate, ql.Quote):
        q_rate_handle = ql.QuoteHandle(forward_rate)
    elif isinstance(forward_rate, float):
        q_rate_handle = ql.QuoteHandle(ql.SimpleQuote(forward_rate))
    else: # Assuming it's already a Handle<Quote>
        q_rate_handle = forward_rate
    return ql.FlatForward(evaluation_date, q_rate_handle, day_counter)

def flat_vol_py(evaluation_date, vol_level, day_counter):
    if isinstance(vol_level, ql.Quote):
        vol_quote_handle = ql.QuoteHandle(vol_level)
    elif isinstance(vol_level, float):
        vol_quote_handle = ql.QuoteHandle(ql.SimpleQuote(vol_level))
    else:
        vol_quote_handle = vol_level
    return ql.BlackConstantVol(evaluation_date, ql.NullCalendar(), vol_quote_handle, day_counter)

def time_to_days_py(t): # As per C++ utilities.hpp
    return int(t * 360 + 0.5)

def exercise_type_to_string_py(exercise):
    if isinstance(exercise, ql.EuropeanExercise):
        return "European"
    elif isinstance(exercise, ql.AmericanExercise):
        return "American"
    return "UnknownExercise"

def payoff_type_to_string_py(payoff):
    if isinstance(payoff, ql.CashOrNothingPayoff):
        return "CashOrNothing"
    elif isinstance(payoff, ql.AssetOrNothingPayoff):
        return "AssetOrNothing"
    elif isinstance(payoff, ql.GapPayoff):
        return "Gap"
    return "UnknownPayoff"


class DigitalOptionData:
    def __init__(self, type, strike, s, q, r, t, v, result, tol, knockin=True):
        self.type = type
        self.strike = strike
        self.s = s
        self.q = q
        self.r = r
        self.t = t
        self.v = v
        self.result = result
        self.tol = tol
        self.knockin = knockin # C++ uses this to switch between AmericanDigital and AmericanDigitalKO

class DigitalOptionTests(unittest.TestCase):

    def setUp(self):
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Set a default evaluation date for tests, can be overridden
        # The C++ test often uses Date::todaysDate() which can vary.
        # For reproducibility, a fixed date is better.
        # Let's use the date from the American Greeks test.
        self.today = ql.Date(15, ql.May, 2007) # Arbitrary fixed date
        ql.Settings.instance().evaluationDate = self.today

        self.dc = ql.Actual360()
        self.spot_q = ql.SimpleQuote(0.0)
        self.q_rate_q = ql.SimpleQuote(0.0)
        self.r_rate_q = ql.SimpleQuote(0.0)
        self.vol_q = ql.SimpleQuote(0.0)

        self.spot_h = ql.QuoteHandle(self.spot_q)
        # qTS_h and rTS_h are Handles to YieldTermStructure
        self.qTS_h = ql.YieldTermStructureHandle(flat_rate_py(self.today, self.q_rate_q, self.dc))
        self.rTS_h = ql.YieldTermStructureHandle(flat_rate_py(self.today, self.r_rate_q, self.dc))
        # volTS_h is Handle to BlackVolTermStructure
        self.volTS_h = ql.BlackVolTermStructureHandle(flat_vol_py(self.today, self.vol_q, self.dc))

        # Process needs to be recreated if market data changes
        self.process = ql.BlackScholesMertonProcess(self.spot_h, self.qTS_h, self.rTS_h, self.volTS_h)

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def _update_market_data(self, s, q, r, v):
        self.spot_q.setValue(s)
        self.q_rate_q.setValue(q)
        self.r_rate_q.setValue(r)
        self.vol_q.setValue(v)
        # Term structures and process will pick up new quote values automatically due to handles

    def _report_failure(self, greek_name, payoff, exercise, s, q, r, today,
                        v, expected, calculated, error, tolerance, knockin):
        msg = (f"{exercise_type_to_string_py(exercise)} "
               f"{payoff.optionType()} option with "
               f"{payoff_type_to_string_py(payoff)} payoff:\n"
               f"    spot value:       {s}\n"
               f"    strike:           {payoff.strike()}\n"
               f"    dividend yield:   {q:.6f}\n"
               f"    risk-free rate:   {r:.6f}\n"
               f"    reference date:   {today}\n"
               f"    maturity:         {exercise.lastDate()}\n"
               f"    volatility:       {v:.6f}\n\n"
               f"    expected   {greek_name}: {expected}\n"
               f"    calculated {greek_name}: {calculated}\n"
               f"    error:            {error}\n"
               f"    tolerance:        {tolerance}\n"
               f"    knock_in:         {knockin}")
        self.fail(msg)

    def test_cash_or_nothing_european_values(self):
        print("Testing European cash-or-nothing digital option...")
        values = [
            DigitalOptionData(ql.Option.Put, 80.00, 100.0, 0.06, 0.06, 0.75, 0.35, 2.6710, 1e-4, True)
        ]
        cash_payoff_amount = 10.0 # As per C++ test

        for val_data in values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)

            payoff = ql.CashOrNothingPayoff(val_data.type, val_data.strike, cash_payoff_amount)
            ex_date = self.today + time_to_days_py(val_data.t)
            exercise = ql.EuropeanExercise(ex_date)

            engine = ql.AnalyticEuropeanEngine(self.process)
            option = ql.VanillaOption(payoff, exercise)
            option.setPricingEngine(engine)

            calculated = option.NPV()
            error = abs(calculated - val_data.result)
            if error > val_data.tol:
                self._report_failure("value", payoff, exercise, val_data.s, val_data.q, val_data.r,
                                     self.today, val_data.v, val_data.result, calculated,
                                     error, val_data.tol, val_data.knockin)

    def test_asset_or_nothing_european_values(self):
        print("Testing European asset-or-nothing digital option...")
        values = [
            DigitalOptionData(ql.Option.Put, 65.00, 70.0, 0.05, 0.07, 0.50, 0.27, 20.2069, 1e-4, True)
        ]
        for val_data in values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)
            payoff = ql.AssetOrNothingPayoff(val_data.type, val_data.strike)
            ex_date = self.today + time_to_days_py(val_data.t)
            exercise = ql.EuropeanExercise(ex_date)
            engine = ql.AnalyticEuropeanEngine(self.process)
            option = ql.VanillaOption(payoff, exercise)
            option.setPricingEngine(engine)
            calculated = option.NPV()
            error = abs(calculated - val_data.result)
            if error > val_data.tol:
                self._report_failure("value", payoff, exercise, val_data.s, val_data.q, val_data.r,
                                     self.today, val_data.v, val_data.result, calculated,
                                     error, val_data.tol, val_data.knockin)

    def test_gap_european_values(self):
        print("Testing European gap digital option...")
        values = [
            DigitalOptionData(ql.Option.Call, 50.00, 50.0, 0.00, 0.09, 0.50, 0.20, -0.0053, 1e-4, True)
        ]
        second_strike = 57.00 # For GapPayoff
        for val_data in values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)
            payoff = ql.GapPayoff(val_data.type, val_data.strike, second_strike)
            ex_date = self.today + time_to_days_py(val_data.t)
            exercise = ql.EuropeanExercise(ex_date)
            engine = ql.AnalyticEuropeanEngine(self.process)
            option = ql.VanillaOption(payoff, exercise)
            option.setPricingEngine(engine)
            calculated = option.NPV()
            error = abs(calculated - val_data.result)
            if error > val_data.tol:
                self._report_failure("value", payoff, exercise, val_data.s, val_data.q, val_data.r,
                                     self.today, val_data.v, val_data.result, calculated,
                                     error, val_data.tol, val_data.knockin)

    def test_cash_at_hit_or_nothing_american_values(self):
        print("Testing American cash-(at-hit)-or-nothing digital option...")
        cash_payoff_val = 15.00
        values = [
            DigitalOptionData(ql.Option.Put, 100.00, 105.00, 0.00, 0.10, 0.5, 0.20,  9.7264, 1e-4, True),
            DigitalOptionData(ql.Option.Call,100.00,  95.00, 0.00, 0.10, 0.5, 0.20, 11.6553, 1e-4, True),
            DigitalOptionData(ql.Option.Call,100.00, 105.00, 0.00, 0.10, 0.5, 0.20, 15.0000, 1e-16,True),
            DigitalOptionData(ql.Option.Put, 100.00,  95.00, 0.00, 0.10, 0.5, 0.20, 15.0000, 1e-16,True),
            DigitalOptionData(ql.Option.Put, 100.00, 105.00, 0.20, 0.10, 0.5, 0.20, 12.2715, 1e-4, True),
            DigitalOptionData(ql.Option.Call,100.00,  95.00, 0.20, 0.10, 0.5, 0.20,  8.9109, 1e-4, True),
            DigitalOptionData(ql.Option.Call,100.00, 105.00, 0.20, 0.10, 0.5, 0.20, 15.0000, 1e-16,True),
            DigitalOptionData(ql.Option.Put, 100.00,  95.00, 0.20, 0.10, 0.5, 0.20, 15.0000, 1e-16,True)
        ]
        for val_data in values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)
            payoff = ql.CashOrNothingPayoff(val_data.type, val_data.strike, cash_payoff_val)
            ex_date = self.today + time_to_days_py(val_data.t)
            # AmericanExercise(earliestDate, latestDate, payoffAtExpiry=False for at-hit)
            am_exercise = ql.AmericanExercise(self.today, ex_date, False)

            engine = ql.AnalyticDigitalAmericanEngine(self.process)
            option = ql.VanillaOption(payoff, am_exercise)
            option.setPricingEngine(engine)

            calculated = option.NPV()
            error = abs(calculated - val_data.result)
            if error > val_data.tol:
                self._report_failure("value", payoff, am_exercise, val_data.s, val_data.q, val_data.r,
                                     self.today, val_data.v, val_data.result, calculated,
                                     error, val_data.tol, val_data.knockin)

    def test_cash_at_expiry_or_nothing_american_values(self):
        print("Testing American cash-(at-expiry)-or-nothing digital option...")
        cash_payoff_val = 15.00
        values = [
            DigitalOptionData(ql.Option.Put,  100.00, 105.00, 0.00, 0.10, 0.5, 0.20,  9.3604, 1e-4, True ),
            DigitalOptionData(ql.Option.Call, 100.00,  95.00, 0.00, 0.10, 0.5, 0.20, 11.2223, 1e-4, True ),
            DigitalOptionData(ql.Option.Put,  100.00, 105.00, 0.00, 0.10, 0.5, 0.20,  4.9081, 1e-4, False), # knock_in = False
            DigitalOptionData(ql.Option.Call, 100.00,  95.00, 0.00, 0.10, 0.5, 0.20,  3.0461, 1e-4, False), # knock_in = False
            DigitalOptionData(ql.Option.Call, 100.00, 105.00, 0.00, 0.10, 0.5, 0.20, 15.0000*math.exp(-0.10*0.5), 1e-12, True),
            DigitalOptionData(ql.Option.Put,  100.00,  95.00, 0.00, 0.10, 0.5, 0.20, 15.0000*math.exp(-0.10*0.5), 1e-12, True),
            DigitalOptionData(ql.Option.Call,   2.37,   2.33, 0.07, 0.43,0.19,0.005,  0.0000, 1e-4, False)
        ]
        for val_data in values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)
            payoff = ql.CashOrNothingPayoff(val_data.type, val_data.strike, cash_payoff_val)
            ex_date = self.today + time_to_days_py(val_data.t)
            # AmericanExercise(earliestDate, latestDate, payoffAtExpiry=True for at-expiry)
            am_exercise_at_expiry = ql.AmericanExercise(self.today, ex_date, True)

            if val_data.knockin: # knockin=true means standard AmDigitalEngine
                engine = ql.AnalyticDigitalAmericanEngine(self.process)
            else: # knockin=false means KOEngine (cash paid at expiry only if NOT hit during life)
                engine = ql.AnalyticDigitalAmericanKOEngine(self.process)

            option = ql.VanillaOption(payoff, am_exercise_at_expiry)
            option.setPricingEngine(engine)

            calculated = option.NPV()
            error = abs(calculated - val_data.result)
            if error > val_data.tol:
                self._report_failure("value", payoff, am_exercise_at_expiry, val_data.s, val_data.q, val_data.r,
                                     self.today, val_data.v, val_data.result, calculated,
                                     error, val_data.tol, val_data.knockin)

    # ... Other tests like AssetAtHit, AssetAtExpiry, Greeks, MC would follow ...
    # For Greeks, the finite difference logic would be:
    def test_cash_at_hit_or_nothing_american_greeks(self):
        print("Testing American cash-(at-hit)-or-nothing digital option greeks...")
        # Setup similar to C++
        # For brevity, will do one iteration

        # Reset eval date for this specific test setup
        today_greeks = ql.Date(20, ql.May, 2008) # Example
        ql.Settings.instance().evaluationDate = today_greeks
        self._update_market_data(100.0, 0.05, 0.05, 0.20) # s, q, r, v

        payoff = ql.CashOrNothingPayoff(ql.Option.Call, 100.0, 100.0) # type, strike, cash
        ex_date = today_greeks + ql.Period(360, ql.Days)
        am_exercise = ql.AmericanExercise(today_greeks, ex_date, False) # At-hit

        # Use distinct process for greeks test to avoid interference
        spot_q_greek = ql.SimpleQuote(100.0)
        q_rate_q_greek = ql.SimpleQuote(0.05)
        r_rate_q_greek = ql.SimpleQuote(0.05)
        vol_q_greek = ql.SimpleQuote(0.20)
        dc_greek = ql.Actual360()

        spot_h_greek = ql.QuoteHandle(spot_q_greek)
        qTS_h_greek = ql.YieldTermStructureHandle(flat_rate_py(today_greeks, q_rate_q_greek, dc_greek))
        rTS_h_greek = ql.YieldTermStructureHandle(flat_rate_py(today_greeks, r_rate_q_greek, dc_greek))
        volTS_h_greek = ql.BlackVolTermStructureHandle(flat_vol_py(today_greeks, vol_q_greek, dc_greek))
        process_greek = ql.BlackScholesMertonProcess(spot_h_greek, qTS_h_greek, rTS_h_greek, volTS_h_greek)

        engine = ql.AnalyticDigitalAmericanEngine(process_greek)
        option = ql.VanillaOption(payoff, am_exercise)
        option.setPricingEngine(engine)

        value = option.NPV()
        calculated_delta = option.delta()
        # ... other greeks ...

        if value > 1e-6: # Only check if value is significant
            # Delta by FD
            du = spot_q_greek.value() * 1.0e-4
            spot_q_greek.setValue(spot_q_greek.value() + du)
            value_p_spot = option.NPV()
            spot_q_greek.setValue(spot_q_greek.value() - 2*du) # u-du
            value_m_spot = option.NPV()
            spot_q_greek.setValue(spot_q_greek.value() + du) # Reset to u
            expected_delta_fd = (value_p_spot - value_m_spot) / (2 * du)

            # Relative error for greeks needs care if expected is near zero
            # C++ uses relativeError(expct, calcl, value);
            error_delta = abs(calculated_delta - expected_delta_fd) / (abs(value) if abs(value)>1e-9 else 1.0) # Avoid div by zero
            tol_delta = 5.0e-5 # As per C++

            if error_delta > tol_delta:
                 self._report_failure("delta", payoff, am_exercise, spot_q_greek.value(),
                                     q_rate_q_greek.value(), r_rate_q_greek.value(), today_greeks,
                                     vol_q_greek.value(), expected_delta_fd, calculated_delta,
                                     error_delta, tol_delta, True) # knockin = True for at-hit tests

    def test_mc_cash_at_hit(self):
        print("Testing Monte Carlo cash-(at-hit)-or-nothing American engine...")
        # This test might be slow due to MC samples.
        # Values are from C++ test
        values = [
            DigitalOptionData(ql.Option.Put,  100.00, 105.00, 0.20, 0.10, 0.5, 0.20, 12.2715, 2e-2, True ), # Increased tol for MC
            DigitalOptionData(ql.Option.Call, 100.00,  95.00, 0.20, 0.10, 0.5, 0.20,  8.9109, 2e-2, True )
        ]
        cash_payoff_val = 15.0
        time_steps_per_year = 90 # As in C++
        # C++: Size requiredSamples = Size(std::pow(2.0, 14)-1);  -> 16383
        required_samples = 2**14 - 1
        # C++: Size maxSamples = 1000000; Not directly translatable to requiredTolerance for Python MC
        # Python MC engines often take `requiredTolerance` or `maxSamples` if `requiredTolerance` is not met.
        # The C++ test uses specific MCDigitalEngine.
        # In Python, ql.MCDigitalEngine might be available. Let's assume it is.
        # Its constructor: MCDigitalEngine(process, timeSteps, timeStepsPerYear, brownianBridge, antitheticVariate,
        #                                requiredSamples, requiredTolerance, maxSamples, seed)

        seed = 1

        for val_data in values:
            self._update_market_data(val_data.s, val_data.q, val_data.r, val_data.v)
            payoff = ql.CashOrNothingPayoff(val_data.type, val_data.strike, cash_payoff_val)
            ex_date = self.today + time_to_days_py(val_data.t)
            am_exercise = ql.AmericanExercise(self.today, ex_date, False) # At-hit

            # MCDigitalEngine (LowDiscrepancy means Sobol or similar)
            # Python might expose specific RNG types or default to one.
            # For now, let's use standard MC if specific LowDiscrepancy version isn't directly available.
            # The name `MCDigitalEngine` exists in Python bindings.
            mc_engine = ql.MCDigitalEngine(self.process,
                                           timeSteps=None, # Let engine decide based on timeStepsPerYear
                                           timeStepsPerYear=time_steps_per_year,
                                           brownianBridge=True,
                                           antitheticVariate=False, # C++ does not specify, default is usually False
                                           requiredSamples=required_samples,
                                           requiredTolerance=1e-3, # Give a tolerance for MC
                                           maxSamples=required_samples * 2, # Allow more if tol not met
                                           seed=seed,
                                           # lowDiscrepancy=True # This might be an option if available
                                          )

            option = ql.VanillaOption(payoff, am_exercise)
            option.setPricingEngine(mc_engine)

            calculated = option.NPV()
            error = abs(calculated - val_data.result)
            if error > val_data.tol: # Using tol from DigitalOptionData
                 self._report_failure("value (MC)", payoff, am_exercise, val_data.s, val_data.q, val_data.r,
                                     self.today, val_data.v, val_data.result, calculated,
                                     error, val_data.tol, val_data.knockin)


if __name__ == '__main__':
    print("Testing QuantLib " + ql.__version__)
    unittest.main(argv=['first-arg-is-ignored'], exit=False)